<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/emotion_classifier_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 필수 라이브러리 임포트
import torch                             # 텐서 연산과 딥러닝을 위한 파이토치
import torch.nn as nn                    # 파이토치의 신경망 모듈
import numpy as np                       # 수치 연산을 위한 NumPy
import re                                # 텍스트 처리를 위한 정규 표현식 (필요한 경우)
import gzip                              # 압축 파일 처리를 위한 라이브러리
import json                              # JSON 데이터 구문 분석을 위한 라이브러리
import requests                          # 데이터 다운로드를 위한 HTTP 요청 라이브러리
import random                            # 데이터 셔플링 및 난수 시드 설정을 위한 라이브러리
import pickle                            # 직렬화된 객체 저장 및 로딩을 위한 라이브러리
import os                                # 파일 시스템 작업을 위한 라이브러리
from tqdm import tqdm                    # 루프 중 진행률 표시를 위한 라이브러리
from torch.utils.data import Dataset, DataLoader  # 파이토치에서 사용자 정의 데이터셋 및 데이터 로더 생성을 위한 라이브러리
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def set_seed(seed):
    """
    다양한 라이브러리에서 재현성을 위해 난수 시드를 설정합니다.
    Args:
        seed (int): 난수 생성을 위한 시드 값.
    """
    random.seed(seed)                      # 파이썬 내장 random 모듈의 시드 설정
    torch.manual_seed(seed)                # 파이토치 CPU 연산을 위한 시드 설정
    torch.cuda.manual_seed_all(seed)       # 모든 GPU에 대한 시드 설정
    torch.backends.cudnn.deterministic = True  # cuDNN에 결정론적 알고리즘 사용
    torch.backends.cudnn.benchmark = False     # 일관된 동작을 위해 cuDNN 자동 튜너 비활성화

class Tokenizer:
    """
    공백을 기준으로 텍스트를 분할하는 기본 토크나이저.
    """
    def tokenize(self, text):
        """
        입력 텍스트를 공백을 기준으로 토큰으로 분할합니다.
        Args:
            text (str): 토큰화할 입력 텍스트 문자열.
        Returns:
            list: 단어 토큰의 목록.
        """
        words = text.split()             # 공백을 기준으로 텍스트 분할
        return words

class Embedder:
    """
    토큰을 해당 임베딩으로 변환하는 클래스.
    Attributes:
        embeddings (dict): 단어에서 벡터 표현으로의 매핑 딕셔너리.
        emb_dim (int): 임베딩의 차원.
        seq_len (int): 입력 텍스트의 고정 시퀀스 길이.
    """
    def __init__(self, embeddings, emb_dim, seq_len):
        """
        임베딩, 임베딩 차원 및 시퀀스 길이로 Embedder를 초기화합니다.
        Args:
            embeddings (dict): 미리 로드된 단어 임베딩.
            emb_dim (int): 임베딩의 차원.
            seq_len (int): 고려할 최대 토큰 수.
        """
        self.embeddings = embeddings
        self.emb_dim = emb_dim
        self.seq_len = seq_len

    def embed(self, tokens):
        """
        토큰 목록을 임베딩 텐서로 변환합니다.
        토큰은 임베딩 딕셔너리에서 조회됩니다. 토큰을 찾을 수 없는 경우,
        영 벡터가 사용됩니다. 시퀀스는 고정 시퀀스 길이에 맞춰 잘리거나 패딩됩니다.
        Args:
            tokens (list): 토큰 문자열 목록.
        Returns:
            torch.Tensor: 임베딩된 토큰을 나타내는 (seq_len, emb_dim) 크기의 텐서.
        """
        embeddings = []
        # 최대 시퀀스 길이까지 각 토큰 처리
        for word in tokens[:self.seq_len]:
            if word in self.embeddings:
                embeddings.append(torch.tensor(self.embeddings[word]))
            elif word.lower() in self.embeddings:
                embeddings.append(torch.tensor(self.embeddings[word.lower()]))
            else:
                # 임베딩에서 찾을 수 없는 단어에는 영 벡터 사용
                embeddings.append(torch.zeros(self.emb_dim))
        # 토큰 수가 seq_len보다 작은 경우 영 벡터로 시퀀스 패딩
        if len(embeddings) < self.seq_len:
            padding_size = self.seq_len - len(embeddings)
            embeddings.extend([torch.zeros(self.emb_dim)] * padding_size)
        # 텐서 목록을 (seq_len, emb_dim) 크기의 단일 텐서로 쌓기
        return torch.stack(embeddings)

def download_file(url, filename, chunk_size=8192):
    if os.path.exists(filename):
        print(f"파일 {filename}이 이미 존재합니다. 다운로드를 건너뜁니다.")
        return

    # 세션 생성 및 재시도 설정
    session = requests.Session()
    retries = Retry(
        total=5,              # 총 5번 재시도
        backoff_factor=1,     # 점점 늘어나는 대기시간 (1s, 2s, 4s…)
        status_forcelist=[500, 502, 503, 504]
    )
    session.mount("http://", HTTPAdapter(max_retries=retries))
    session.mount("https://", HTTPAdapter(max_retries=retries))

    with session.get(url, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get("content-length", 0))

        with tqdm(total=total_size, unit="B", unit_scale=True, unit_divisor=1024, desc="다운로드 중") as progress_bar:
            with open(filename, "wb") as f:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    if chunk:
                        f.write(chunk)
                        progress_bar.update(len(chunk))

    print(f"다운로드 완료: {filename}")

def load_embeddings(url, filename="vectors.dat"):
    """
    gzip 압축 파일에서 단어 임베딩을 다운로드하고 로드합니다.
    파일이 로컬에 존재하지 않는 경우, 제공된 URL에서 다운로드됩니다.
    파일은 어휘 크기와 임베딩 차원을 나타내는 헤더를 가질 것으로 예상되며,
    그 뒤에 각 단어와 해당 바이너리 벡터가 이어집니다.
    Args:
        url (str): 임베딩을 다운로드할 URL.
        filename (str): 임베딩을 저장/로드할 로컬 파일명.
    Returns:
        tuple: 다음을 포함하는 튜플:
            - vectors (dict): 단어에서 임베딩 벡터(NumPy 배열로)로의 매핑.
            - emb_dim (int): 임베딩 벡터의 차원.
    """
    # 임베딩 파일이 로컬에 존재하는지 확인한 후 다운로드
    download_file(url, filename, chunk_size=1024)

    # 바이너리 읽기 모드로 gzip 압축 임베딩 파일 열기
    with gzip.open(filename, "rb") as f:
        # 헤더 라인을 읽어 어휘 크기와 임베딩 차원 가져오기
        header = f.readline()
        vocab_size, emb_dim = map(int, header.split())
        vectors = {}
        # 각 임베딩 벡터의 바이트 수 계산
        binary_len = np.dtype("float32").itemsize * emb_dim
        # 진행률 표시줄과 함께 각 단어와 해당 임베딩 벡터 읽기
        with tqdm(total=vocab_size, desc="단어 벡터 로딩 중") as pbar:
            for _ in range(vocab_size):
                word = []
                # 공백이 나타날 때까지(단어 끝을 나타냄) 문자를 하나씩 읽기
                while True:
                    ch = f.read(1)
                    if ch == b" ":
                        word = b"".join(word).decode("utf-8")
                        break
                    if ch != b"\n":
                        word.append(ch)
                # 바이너리 벡터 데이터를 읽고 float32 유형의 NumPy 배열로 변환
                vector = np.frombuffer(f.read(binary_len), dtype="float32")
                vectors[word] = vector
                pbar.update(1)
    return vectors, emb_dim

def load_and_split_data(url, test_ratio=0.1):
    """
    데이터셋을 다운로드, 압축 해제 및 훈련 및 테스트 세트로 분할합니다.
    데이터셋은 각 라인이 JSON 객체인 gzip 파일로 예상됩니다.
    Args:
        url (str): 데이터셋을 다운로드할 URL.
        test_ratio (float): 테스트 세트로 사용할 데이터 비율.
    Returns:
        tuple: 다음을 포함하는 튜플:
            - train_data (list): 훈련 예시 목록.
            - test_data (list): 테스트 예시 목록.
    """
    # 제공된 URL에서 데이터셋 다운로드
    response = requests.get(url)
    # gzip 압축 콘텐츠 압축 해제 및 문자열로 디코딩
    content = gzip.decompress(response.content).decode()
    # 각 라인을 JSON 객체로 구문 분석
    data = [json.loads(line) for line in content.splitlines()]
    # 무작위 분포를 보장하기 위해 데이터 셔플
    random.shuffle(data)
    # test_ratio를 기반으로 분할 인덱스 결정
    split_index = int(len(data) * (1 - test_ratio))
    return data[:split_index], data[split_index:]

def download_and_prepare_data(data_url, vectors_url, seq_len, batch_size):
    """
    훈련 및 평가를 위해 데이터셋과 단어 임베딩을 다운로드하고 준비합니다.
    이 함수는 텍스트 데이터셋과 단어 임베딩을 다운로드하고, 레이블 매핑을 생성하며,
    토크나이저와 Embedder를 초기화하고, 훈련 및 테스트를 위한 데이터 로더를 반환합니다.
    Args:
        data_url (str): 텍스트 데이터셋을 다운로드할 URL.
        vectors_url (str): 단어 임베딩을 다운로드할 URL.
        seq_len (int): 토큰 임베딩의 고정 시퀀스 길이.
        batch_size (int): 데이터 로더의 배치 크기.
    Returns:
        tuple: 다음을 포함하는 튜플:
            - train_loader (DataLoader): 훈련 데이터용 DataLoader.
            - test_loader (DataLoader): 테스트 데이터용 DataLoader.
            - id_to_label (dict): 레이블 ID에서 레이블 이름으로의 매핑.
            - num_classes (int): 고유 클래스 수.
            - emb_dim (int): 단어 임베딩의 차원.
    """
    # 데이터셋을 훈련 및 테스트 분할로 로드 및 분할
    train_split, test_split = load_and_split_data(data_url, test_ratio=0.1)
    # 사전 훈련된 단어 임베딩 로드 및 임베딩 차원 가져오기
    embeddings, emb_dim = load_embeddings(vectors_url)
    # 훈련 데이터를 사용하여 레이블과 숫자 ID 사이의 매핑 생성
    label_to_id, id_to_label, num_classes = create_label_mappings(train_split)
    # 로드된 임베딩과 시퀀스 길이로 토크나이저와 Embedder 초기화
    tokenizer = Tokenizer()
    embedder = Embedder(embeddings, emb_dim, seq_len)
    # 훈련 및 테스트 데이터셋 모두에 대한 DataLoader 생성
    train_loader, test_loader = create_data_loaders(
        train_split, test_split,
        tokenizer, embedder,
        label_to_id, batch_size
    )
    return (train_loader, test_loader, id_to_label, num_classes, emb_dim)

class TextClassificationDataset(Dataset):
    """
    텍스트 분류를 위한 파이토치 데이터셋.
    이 데이터셋은 원시 텍스트와 레이블 데이터를 모델에 공급할 수 있는 형식으로 변환합니다.
    제공된 토크나이저와 Embedder를 사용하여 텍스트를 토큰화한 다음 임베딩합니다.
    Args:
        data (list): "text" 및 "label" 키를 포함하는 딕셔너리 리스트.
        tokenizer (Tokenizer): 텍스트를 토큰으로 분할하는 토크나이저 인스턴스.
        embedder (Embedder): 토큰을 임베딩으로 변환하는 Embedder 인스턴스.
        label_to_id (dict): 레이블 문자열에서 숫자 ID로의 매핑.
    """
    def __init__(self, data, tokenizer, embedder, label_to_id):
        # 텍스트 추출 및 레이블을 해당 ID로 변환
        self.texts = [item["text"] for item in data]
        self.label_ids = [label_to_id[item["label"]] for item in data]
        self.tokenizer = tokenizer
        self.embedder = embedder

    def __len__(self):
        """
        데이터셋의 총 예시 수를 반환합니다.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        지정된 인덱스의 예시에 대한 임베딩된 텍스트와 레이블을 검색합니다.
        Args:
            idx (int): 검색할 예시의 인덱스.
        Returns:
            tuple: 다음을 포함하는 튜플:
                - embeddings (torch.Tensor): (seq_len, emb_dim) 크기의 텐서.
                - label (torch.Tensor): 레이블 ID를 포함하는 텐서.
        """
        # 주어진 인덱스의 텍스트 토큰화
        tokens = self.tokenizer.tokenize(self.texts[idx])
        # Embedder를 사용하여 토큰을 임베딩으로 변환
        embeddings = self.embedder.embed(tokens)
        # 임베딩과 해당 레이블을 텐서로 반환
        return embeddings, torch.tensor(self.label_ids[idx], dtype=torch.long)

class CNNTextClassifier(nn.Module):
    """
    텍스트 분류를 위한 합성곱 신경망.
    이 모델은 입력 텍스트의 임베딩된 표현을 기반으로 분류하기 위해
    두 개의 1D 합성곱 층과 그 뒤에 완전 연결 층을 적용합니다.
    Args:
        emb_dim (int): 입력 임베딩의 차원.
        num_classes (int): 분류를 위한 타깃 클래스 수.
        seq_len (int): 입력 텍스트의 고정 시퀀스 길이.
        id_to_label (dict): 레이블 ID에서 레이블 이름으로의 매핑.
    """
    def __init__(self, emb_dim, num_classes, seq_len, id_to_label):
        super().__init__()
        # 나중에 사용하기 위해 모델 구성 저장 (예: 추론 중)
        self.config = {
            "emb_dim": emb_dim,
            "num_classes": num_classes,
            "seq_len": seq_len,
            "id_to_label": id_to_label
        }
        # 첫 번째 합성곱 층: 입력 채널 = emb_dim, 출력 채널 = 512
        self.conv1 = nn.Conv1d(emb_dim, 512, kernel_size=3, padding=1)
        # 두 번째 합성곱 층: 입력 채널 = 512, 출력 채널 = 256
        self.conv2 = nn.Conv1d(512, 256, kernel_size=3, padding=1)
        # 펼쳐진 특성을 128 단위로 줄이는 완전 연결 층
        self.fc1 = nn.Linear(256 * seq_len, 128)
        # 클래스 수에 매핑하는 출력 층
        self.fc2 = nn.Linear(128, num_classes)
        # ReLU 활성화 함수
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        CNN 모델의 포워드 패스를 정의합니다.
        Args:
            x (torch.Tensor): (batch_size, seq_len, emb_dim) 크기의 입력 텐서.
        Returns:
            torch.Tensor: (batch_size, num_classes) 크기의 출력 로짓 텐서.
        """
        x = x.permute(0, 2, 1)           # (batch_size, emb_dim, seq_len)로 차원 재배열
        x = self.relu(self.conv1(x))     # 첫 번째 합성곱 및 ReLU 활성화 적용
        x = self.relu(self.conv2(x))     # 두 번째 합성곱 및 ReLU 활성화 적용
        x = x.flatten(start_dim=1)       # 완전 연결 층을 위해 특성 펼치기
        x = self.fc1(x)                  # 첫 번째 완전 연결 층 적용
        return self.fc2(x)               # 최종 층에서 출력 로짓 반환

def calculate_accuracy(model, dataloader, device):
    """
    제공된 데이터셋에서 모델의 정확도를 평가합니다.
    Args:
        model (nn.Module): 훈련된 모델.
        dataloader (DataLoader): 평가할 데이터셋의 DataLoader.
        device: 연산이 수행되는 장치.
    Returns:
        float: 정확한 예측의 비율로서의 정확도.
    """
    model.eval()                       # 모델을 평가 모드로 설정
    correct = 0
    total = 0
    with torch.no_grad():              # 효율성을 위해 그레이디언트 계산 비활성화
        for batch in dataloader:
            embeddings, labels = batch
            embeddings = embeddings.to(device)
            labels = labels.to(device)
            outputs = model(embeddings)           # 포워드 패스
            _, predicted = torch.max(outputs, 1)   # 예측된 클래스 인덱스 가져오기
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    model.train()                      # 모델을 훈련 모드로 다시 설정
    return accuracy

def create_label_mappings(train_dataset):
    """
    훈련 데이터셋을 기반으로 레이블 문자열과 숫자 ID 사이의 매핑을 생성합니다.
    Args:
        train_dataset (list): "label" 필드가 있는 훈련 예시 목록.
    Returns:
        tuple: 다음을 포함하는 튜플:
            - label_to_id (dict): 레이블 문자열에서 숫자 ID로의 매핑.
            - id_to_label (dict): 숫자 ID에서 레이블 문자열로의 매핑.
            - num_classes (int): 고유 클래스의 총 수.
    """
    # 훈련 데이터에서 고유 레이블 추출 및 정렬
    unique_labels = sorted(set(item["label"] for item in train_dataset))
    # 각 레이블을 고유 정수 ID에 매핑
    label_to_id = {label: i for i, label in enumerate(unique_labels)}
    # ID에서 레이블로의 역매핑 생성
    id_to_label = {i: label for label, i in label_to_id.items()}
    return label_to_id, id_to_label, len(unique_labels)

def create_data_loaders(train_split, test_split, tokenizer, embedder, label_to_id, batch_size):
    """
    훈련 및 테스트 데이터셋을 위한 파이토치 DataLoader를 생성합니다.
    Args:
        train_split (list): 훈련 예시 목록.
        test_split (list): 테스트 예시 목록.
        tokenizer (Tokenizer): 텍스트 처리를 위한 토크나이저 인스턴스.
        embedder (Embedder): 토큰을 임베딩으로 변환하기 위한 Embedder 인스턴스.
        label_to_id (dict): 레이블 문자열에서 숫자 ID로의 매핑.
        batch_size (int): DataLoader의 배치 크기.
    Returns:
        tuple: 다음을 포함하는 튜플:
            - train_loader (DataLoader): 훈련 데이터셋의 DataLoader.
            - test_loader (DataLoader): 테스트 데이터셋의 DataLoader.
    """
    # 훈련 및 테스트 데이터에 대한 사용자 정의 데이터셋 초기화
    train_dataset = TextClassificationDataset(train_split, tokenizer, embedder, label_to_id)
    test_dataset = TextClassificationDataset(test_split, tokenizer, embedder, label_to_id)
    # 훈련 데이터에 대한 셔플링을 활성화하여 DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size)
    return train_loader, test_loader

def save_model(model, prefix):
    """
    훈련된 모델의 상태 딕셔너리와 구성을 디스크에 저장합니다.
    Args:
        model (nn.Module): 저장할 훈련된 모델.
        prefix (str): 저장된 파일 이름의 접두사.
    """
    # 모델의 상태와 구성을 체크포인트 파일로 저장
    torch.save({
        "state_dict": model.state_dict(),
        "config": model.config
    }, f"{prefix}_model.pth")

def load_model(prefix):
    """
    디스크에서 저장된 모델을 로드하고 평가를 위해 준비합니다.
    Args:
        prefix (str): 모델 저장 중 사용된 접두사.
    Returns:
        nn.Module: 평가 모드의 로드된 CNNTextClassifier 모델.
    """
    # 모델 상태와 구성을 포함하는 체크포인트 로드
    checkpoint = torch.load(f"{prefix}_model.pth", map_location=torch.device("cpu"))
    config = checkpoint["config"]
    # 저장된 구성을 사용하여 모델 재초기화
    model = CNNTextClassifier(
        emb_dim=config["emb_dim"],
        num_classes=config["num_classes"],
        seq_len=config["seq_len"],
        id_to_label=config["id_to_label"]
    )
    # 저장된 가중치를 모델에 로드
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()  # 모델을 평가 모드로 설정
    return model

def test_model(model, test_input, tokenizer=None, embedder=None):
    """
    단일 입력 텍스트에서 모델을 테스트하고 예측된 레이블을 출력합니다.
    Args:
        model (nn.Module): 훈련된 텍스트 분류 모델.
        test_input (str): 분류할 입력 텍스트.
        tokenizer (Tokenizer, optional): 토크나이저 인스턴스. None인 경우 새로 생성됩니다.
        embedder (Embedder, optional): Embedder 인스턴스. None인 경우 임베딩이 로드되고 새 Embedder 객체가 생성됩니다.
    Notes:
        이 함수는 입력 텍스트와 함께 예측된 감정을 출력합니다.
    """
    # 제공되지 않은 경우 토크나이저 초기화
    if not tokenizer:
        tokenizer = Tokenizer()
    # 제공되지 않은 경우 전역 vectors_url과 seq_len을 사용하여 임베딩 로드 및 Embedder 초기화
    if not embedder:
        embeddings, emb_dim = load_embeddings(vectors_url)
        embedder = Embedder(embeddings, emb_dim, seq_len)
    # 모델의 매개변수에서 장치 결정
    device = next(model.parameters()).device
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():
        # 입력 텍스트 토큰화
        tokens = tokenizer.tokenize(test_input)
        # 토큰을 임베딩으로 변환
        embeddings = embedder.embed(tokens)
        # 임베딩을 float 텐서로 변환, 배치 차원 추가, 올바른 장치로 이동
        embeddings = torch.tensor(embeddings, dtype=torch.float32).unsqueeze(0).to(device)
        # 모델을 통한 포워드 패스로 예측 가져오기
        outputs = model(embeddings)
        _, predicted = torch.max(outputs.data, 1)
        # 예측된 숫자 레이블을 실제 레이블 문자열로 매핑
        predicted_label = model.config["id_to_label"][predicted.item()]
    print(f"입력: {test_input}")
    print(f"예측된 감정: {predicted_label}")

def set_hyperparameters():
    """
    훈련을 위한 하이퍼파라미터를 정의하고 반환합니다.
    Returns:
        tuple: 다음을 포함하는 튜플:
            - num_epochs (int): 훈련 에포크 수.
            - seq_len (int): 입력 텍스트의 고정 시퀀스 길이.
            - batch_size (int): 훈련 배치 크기.
            - learning_rate (float): 옵티마이저의 학습률.
    """
    num_epochs = 2
    seq_len = 100
    batch_size = 32
    learning_rate = 0.001
    return num_epochs, seq_len, batch_size, learning_rate

In [6]:
# 재현성을 위해 난수 시드 설정
set_seed(42)
# 계산 장치 결정: 사용 가능한 경우 GPU 사용, 그렇지 않으면 CPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")

# 데이터셋과 단어 임베딩 다운로드를 위한 URL
data_url = "https://www.thelmbook.com/data/emotions"
vectors_url = "https://www.thelmbook.com/data/word-vectors"

# 훈련 하이퍼파라미터 설정
num_epochs, seq_len, batch_size, learning_rate = set_hyperparameters()

# 데이터 로더, 레이블 매핑, 임베딩 차원 다운로드 및 준비
train_loader, test_loader, id_to_label, num_classes, emb_dim = \
    download_and_prepare_data(data_url, vectors_url, seq_len, batch_size)

# 임베딩 차원과 레이블 매핑으로 CNN 텍스트 분류기 모델 초기화
model = CNNTextClassifier(emb_dim, num_classes, seq_len, id_to_label)
model = model.to(device)  # 모델을 적절한 장치로 이동

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()                # 분류를 위한 교차 엔트로피 손실
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)  # AdamW 옵티마이저

# 여러 에포크에 대한 훈련 루프
for epoch in range(num_epochs):
    model.train()                  # 모델을 훈련 모드로 설정
    total_loss = 0
    num_batches = 0
    # 현재 에포크에 대한 진행률 표시줄 초기화
    progress_bar = tqdm(train_loader, desc=f"에포크 {epoch+1}/{num_epochs}")
    for batch in progress_bar:
        batch_embeddings, batch_labels = batch
        batch_embeddings = batch_embeddings.to(device)
        batch_labels = batch_labels.to(device)
        optimizer.zero_grad()      # 역전파 전 그레이디언트 재설정
        outputs = model(batch_embeddings)   # 모델을 통한 포워드 패스
        loss = criterion(outputs, batch_labels)  # 손실 계산
        loss.backward()            # 역전파
        optimizer.step()           # 모델 매개변수 업데이트

        total_loss += loss.item()
        num_batches += 1

        # 현재 평균 손실로 진행률 표시줄 업데이트
        progress_bar.set_postfix({"손실": total_loss / num_batches})

    avg_loss = total_loss / num_batches

    # 각 에포크 후 테스트 세트에서 모델 정확도 평가
    test_acc = calculate_accuracy(model, test_loader, device)
    print(f"에포크 [{epoch+1}/{num_epochs}], 테스트 정확도: {test_acc:.4f}")

# 지정된 접두사로 훈련된 모델 디스크에 저장
model_name = "CNN_classifier"
save_model(model, model_name)

사용 중인 장치: cuda


다운로드 중: 100%|██████████| 1.53G/1.53G [01:39<00:00, 16.5MB/s]


다운로드 완료: vectors.dat


에포크 1/2: 100%|██████████| 563/563 [00:07<00:00, 76.18it/s, 손실=0.761]


에포크 [1/2], 테스트 정확도: 0.8750


에포크 2/2: 100%|██████████| 563/563 [00:05<00:00, 94.79it/s, 손실=0.253]


에포크 [2/2], 테스트 정확도: 0.8995


In [8]:
# 지정된 모델 이름 접두사를 사용하여 이전에 저장된 모델을 로드합니다.
# 'load_model' 함수는 체크포인트를 읽고 CNNTextClassifier를 재구성합니다.
loaded_model = load_model(model_name)

# 제공된 URL에서 단어 임베딩을 로드합니다.
# 이 함수는 단어에서 임베딩 벡터로의 매핑 사전과 임베딩 차원을 반환합니다.
embeddings, emb_dim = load_embeddings(vectors_url)

# 토크나이저를 초기화합니다.
# 여기서 Tokenizer는 공백을 기준으로 텍스트를 토큰으로 분할하는 간단한 역할을 합니다.
tokenizer = Tokenizer()

# 로드된 임베딩, 임베딩 차원 및 고정 시퀀스 길이를 사용하여 Embedder 인스턴스를 생성합니다.
# Embedder는 토큰화된 텍스트를 모델에 적합한 임베딩 텐서로 변환합니다.
embedder = Embedder(embeddings, emb_dim, seq_len)

# 분류할 샘플 입력 텍스트를 정의합니다.
test_input = "I'm so happy to be able to train a text classifier!"

# test_model 함수를 사용하여 샘플 입력에서 로드된 모델을 평가합니다.
# 이 함수는 텍스트를 토큰화하고, 임베딩으로 변환하며, 모델을 통한 포워드 패스를 수행하고,
# 예측된 레이블을 출력합니다.
test_model(loaded_model, test_input, tokenizer, embedder)

파일 vectors.dat이 이미 존재합니다. 다운로드를 건너뜁니다.


단어 벡터 로딩 중: 100%|██████████| 3000000/3000000 [00:53<00:00, 55747.19it/s]
/tmp/ipython-input-2742773537.py:468: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings = torch.tensor(embeddings, dtype=torch.float32).unsqueeze(0).to(device)


입력: I'm so happy to be able to train a text classifier!
예측된 감정: joy
